This is my Peer-graded Assignment for Segmenting and Clustering Neighborhoods in Toronto

In [13]:
!pip install bs4
#!pip install requests
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd
print("Done importing.")

Done importing.


The following cell is for webscraping.

In [38]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'

#find a html table in the web page
tables = soup.find_all('table') # in html table is represented by the tag <table>
for index,table in enumerate(tables):
    if ("Neighbourhood" in str(table)):
        table_index = index
#print(tables[table_index].prettify())


The following cell is to convert the data into a pandas dataframe.

In [57]:
dfpostal = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        PostalCode = col[0].text.strip()
        Borough= col[1].text.strip()
        if Neighborhood == "Not assigned":
            Neighborhood = Borough
        else:
            Neighborhood = col[2].text.strip()
        if Borough != "Not assigned":
            dfpostal = dfpostal.append({"PostalCode":PostalCode, "Borough":Borough, "Neighborhood":Neighborhood}, ignore_index=True)

dfpostal.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,North York
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Etobicoke
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,North York
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [40]:
dfpostal.shape

(103, 3)

This is to get the geographical coordinates of each postal code.

In [41]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [58]:
dflocation = pd.read_csv('Geospatial_data.csv')
dflocation.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
dflocation.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


This is to merge the dataframes.

In [59]:
dfpostalloc = pd.merge(dfpostal, dflocation, 
                     left_on = 'PostalCode', 
                     right_on = 'PostalCode', 
                     how='left')

dfpostalloc.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,North York,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Etobicoke,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,North York,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
